### Note
* Instructions from the original assignment have been kept in the notebook.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

In [2]:
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [8]:
# each SN is assumed to be one player so the number of players is the number of unique SNs
# the function .nunique() returns the total number of unique values in a column
player_count = purchase_data["SN"].nunique()
player_count


576

## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [11]:
# the .nunique() function was used to determine the number of unique games
unique_items = purchase_data["Item Name"].nunique()

# the number of items sold was determined by counting the items in the Item Name column
sold_count = purchase_data["Item Name"].count()

# average price was determined by taking the average of the values in the Price column using the .mean() function
avg_price = purchase_data["Price"].mean()

# the items in the Price column was summed using the .sum() function
total_revenue = purchase_data["Price"].sum()

# the calculated values were added to a list, which were converted to a data frame using pd.DataFrame()
purchasing_analysis = [{"No. of Unique Items": unique_items, "Avg. Price": avg_price, "No. of Purchases": sold_count, "Total Revenue": total_revenue}]
purchasing_analysis_df = pd.DataFrame(purchasing_analysis)

# reformatting the data values by creating a formatting dictionary
pd.options.display.float_format = '{:.2f}'.format
format_dict = {"Avg. Price":"${0:,.2f}", "Total Revenue": "${0:,.2f}"}

# print summary data frame
purchasing_analysis_df.style.format(format_dict).hide_index()

No. of Unique Items,Avg. Price,No. of Purchases,Total Revenue
179,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [12]:
# create new data frame with a single entry for each unique SN
unique_SN_df = purchase_data.drop_duplicates(subset=["SN"])

# calculate the count of players by gender
gender_count = unique_SN_df['Gender'].value_counts(sort=True, ascending=True)
gender_count_df = pd.DataFrame(gender_count)

# reformat this df so there are column headings (check indices)
gender_count_df.columns = ["Count"]
gender_count_df.index.name = "Gender"

# calculate the percentage of players by gender
gender_percentage= unique_SN_df['Gender'].value_counts(normalize=True)*100
gender_percentage_df = pd.DataFrame(gender_percentage)
gender_percentage_df.index.name = "Gender"
gender_percentage_df.columns = ["Percentage"]

# join data frames
merge_gender_df = pd.merge(gender_count_df, gender_percentage_df, on="Gender")

# reformatting the data values by creating a formatting dictionary
pd.options.display.float_format = '{:.2f}'.format
format_dict = {"Percentage": "{0:,.2f}%"}

# print gender data frame
merge_gender_df.style.format(format_dict)

,Count,Percentage
Gender,,
Other / Non-Disclosed,11,1.91%
Female,81,14.06%
Male,484,84.03%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [38]:
#use groupby to group purchase data by gender
grouped_gender_df = purchase_data.groupby(['Gender'])

#use groupby and count to get purchase count
gender_purchase_count = grouped_gender_df["Purchase ID"].count()

#use groupby and mean to get avg. purchase price
gender_avg_price = grouped_gender_df["Price"].mean()

#use groupby and sum to get total purchase price
gender_tot_price = grouped_gender_df["Price"].sum()

#use groupby of SN to get total purchases per person 
grouped_SN_df = purchase_data.groupby(['SN'])

#sum all purchases for each SN
purchase_person = grouped_SN_df["Price"].sum()

#add total purchases for each SN to original data frame without SN repeats
merge_df = pd.merge(unique_SN_df, purchase_person, on="SN")
merge_df = merge_df.rename(columns={"Price_x":"Price", "Price_y":"Total Purchases"})

#group the merged dataframe by gender
grouped_merge_df = merge_df.groupby(['Gender'])

#calculate average purchase per person using mean 
avg_tot_person = grouped_merge_df["Total Purchases"].mean()

#create a new data frame with the calculated group values
gender_summary_df = pd.DataFrame({"Purchase Count": gender_purchase_count, "Avg. Purchase Price":gender_avg_price,
                                "Total Purchase Value": gender_tot_price, "Avg. Total Purchase per Person":avg_tot_person})

# reformatting the data values by creating a formatting dictionary
pd.options.display.float_format = '{:.2f}'.format
format_dict = {"Avg. Purchase Price": "${0:,.2f}", "Total Purchase Value": "${0:,.2f}","Avg. Total Purchase per Person": "${0:,.2f}"}

# print gender data frame
gender_summary_df.style.format(format_dict)


,Purchase Count,Avg. Purchase Price,Total Purchase Value,Avg. Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [15]:
# Create the bins in which Data will be held
# Bins are <10, 10-14, 15-19, 20-24, 25-29, 30-34,35-39,40+.   
bins = [0, 9, 14, 19, 24, 29,34,39,100]

# Create the names for the five bins
group_labels = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]

# Slice the data and place it into bins
pd.cut(unique_SN_df["Age"], bins, labels=group_labels).head(10)

# Place the data series into a new column inside of the DataFrame with unique SN to avoid double counting 
#of repeat purchasers
#(comment out if re-running)
unique_SN_df["Age Group"] = pd.cut(unique_SN_df["Age"], bins, labels=group_labels)

# Create a GroupBy object based on "Age Group" 
age_group = unique_SN_df.groupby("Age Group")

# count how many players are in each bin
count = age_group["Age"].count()

# calculate the percentage of players are in each bin
percent = (count/player_count)*100

# create a dataframe of binned values
age_summary_df=pd.DataFrame({"Count":count, "Percentage of Players":percent})

# reformatting the data values by creating a formatting dictionary
pd.options.display.float_format = '{:.2f}'.format
format_dict = {"Percentage of Players": "{0:,.2f}%"}

# print gender data frame
age_summary_df.style.format(format_dict)

C:\Users\linds\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Count,Percentage Players
Age Group,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [39]:
# using bins added from previous exercise and updating the original dataframe
# Place the data series into a new column inside of the original dataFrame to include all purchase information
#(comment out if re-running)

purchase_data["Age Group"] = pd.cut(purchase_data["Age"], bins, labels=group_labels)
purchase_data.head(10)

# Create a GroupBy object based on Age Group 
age_group_2 = purchase_data.groupby("Age Group")

#use groupby and count to get purchase count
age_purchase_count = age_group_2["Purchase ID"].count()

#use groupby and mean to get avg. purchase price
age_avg_price = age_group_2["Price"].mean()

#use groupby and sum to get total purchase price
age_tot_price = age_group_2["Price"].sum()


#group the previously merged dataframe by Age Group
grouped_merge_2_df = merge_df.groupby(['Age Group'])

#calculate average purchase per person using mean 
avg_tot_person = grouped_merge_2_df["Total Purchases"].mean()


#create a new data frame with the calculated group values
age_summary_2_df = pd.DataFrame({"Purchase Count": age_purchase_count, "Avg. Purchase Price":age_avg_price,
                                "Total Purchase Value": age_tot_price, "Avg. Total Purchase per Person":avg_tot_person})

# reformatting the data values by creating a formatting dictionary
pd.options.display.float_format = '{:.2f}'.format
format_dict = {"Avg. Purchase Price": "${0:,.2f}", "Total Purchase Value": "${0:,.2f}","Avg. Total Purchase per Person": "${0:,.2f}"}

# print age data frame
age_summary_2_df.style.format(format_dict)

,Purchase Count,Avg. Purchase Price,Total Purchase Value,Avg. Total Purchase per Person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [20]:
#groupby SN to determine individual user information
SN_group = purchase_data.groupby("SN")

#calculate counts
SN_purchase_count = SN_group["Purchase ID"].count()

# calculate avg purchase price
SN_avg_price = SN_group["Price"].mean()

# calculate total purchase value
SN_tot_price = SN_group["Price"].sum()

# create SN summary dataframe
SN_summary_df = pd.DataFrame({"Purchase Count": SN_purchase_count, "Avg. Purchase Price":SN_avg_price,
                                "Total Purchase Value": SN_tot_price })

# sort dataframe by descending total purchase value
sorted_SN_df=SN_summary_df.sort_values(['Total Purchase Value'], ascending=False)

# reformatting the data values by creating a formatting dictionary
pd.options.display.float_format = '{:.2f}'.format
format_dict = {"Avg. Purchase Price": "${0:,.2f}", "Total Purchase Value": "${0:,.2f}"}

# print age data frame
sorted_SN_df.style.format(format_dict)

,Purchase Count,Avg. Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10
Ilarin91,3,$4.23,$12.70
Ialallo29,3,$3.95,$11.84
Tyidaim51,3,$3.94,$11.83
Lassilsala30,3,$3.84,$11.51


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [24]:
#groupby Item ID and Item Name
item_name_group = purchase_data.groupby(["Item ID","Item Name"])

#calculate counts
IN_purchase_count = item_name_group["Purchase ID"].count()

#calculate average item price
IN_avg_price = item_name_group["Price"].mean()

# calculate total purchase value
IN_tot_price = item_name_group["Price"].sum()

# create summary dataframe
IN_summary_df = pd.DataFrame({"Purchase Count": IN_purchase_count, "Avg. Item Price":IN_avg_price, "Total Purchase Value": IN_tot_price})

# sort summary dataframe by purchase count in descending order
sorted_IN_summary_df = IN_summary_df.sort_values('Purchase Count', ascending=False)

# reformatting the data values by creating a formatting dictionary
pd.options.display.float_format = '{:.2f}'.format
format_dict = {"Avg. Item Price": "${0:,.2f}", "Total Purchase Value": "${0:,.2f}"}

# print age data frame
sorted_IN_summary_df.style.format(format_dict)

,,Purchase Count,Avg. Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
75,Brutality Ivory Warmace,8,$2.42,$19.36
103,Singed Scalpel,8,$4.35,$34.80
34,Retribution Axe,8,$2.22,$17.76


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [25]:
# sort summary dataframe by total purchase value in descending order
sorted_total_df = sorted_IN_summary_df.sort_values('Total Purchase Value', ascending=False)

# reformatting the data values by creating a formatting dictionary
pd.options.display.float_format = '{:.2f}'.format
format_dict = {"Avg. Item Price": "${0:,.2f}", "Total Purchase Value": "${0:,.2f}"}

# print age data frame
sorted_total_df.style.format(format_dict)

,,Purchase Count,Avg. Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
59,"Lightning, Etcher of the King",8,$4.23,$33.84
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
78,"Glimmer, Ender of the Moon",7,$4.40,$30.80
72,Winter's Bite,8,$3.77,$30.16


# Summary of observable trends in Heroes of Pymoli data

* Approximately 50% of the total revenue came from players ages 20-24. 
* More than 50% of the total revenue came from male players.
* The average total purchase per person varied slightly by age and gender (~$0.50). The minority gender demographic (female and other/non-disclosed) and the majority age demographic (ages 20-24) had higher average total purchases per person. Additionally, the minority gender demographic had a slightly higher average purchase price while the majority age demographic had a slightly lower average purchase price. This would suggest increased revenue with increasing numbers of female/other/non-disclosed players.